In [3]:
import pandas as pd
from collections import Counter
import string

In [4]:
data = pd.read_csv("data.csv", sep=";;;", engine="python")

In [119]:
def df_of_counter(cnt, *args, **kwargs):
    # takes collections.Counter of and returns pandas.DataFrame where first column is the keys and second column is the values
    
    column_1 = list(cnt.keys())
    column_2 = list(cnt.values())


    return pd.DataFrame(list(zip(column_1, column_2)), *args, **kwargs)

## Nestandardni znaki v imenih
Ker so me pri zajemu podatkov zelo mučili nestandardni znaki ki jih ljudje postavljajo v svoje oglase si najprej oglejmo te

In [105]:
c = Counter()
for ime in data.Name.to_list():
    for char in ime:
        # Ignorirajmo dolgočasne alfanumerične znake
        if char not in string.ascii_letters and char not in string.digits:
            c[char] += 1

In [120]:
df = df_of_counter(c, columns=['CHARACTER', 'FREQUENCY']).sort_values('FREQUENCY', ascending=False).reset_index(drop=True)
df

,CHARACTER,FREQUENCY
0,,60200
1,",",4581
2,-,3738
3,/,3730
4,č,2018
5,.,1350
6,(,1087
7,),1055
8,+,905
9,š,657


## Mesta prodaje
Zdaj si oglejmo iz katerega mesta je oddanih največ oglasov

In [126]:
d = Counter()
for location in data.City.to_list():
    if len(l := location.split(", ")) == 2:
        city, _ = l
    else:
        city = l[0]
    d[city] += 1

In [131]:
df = df_of_counter(d, columns=["CITY", "N_OF_ADDS"]).sort_values('N_OF_ADDS', ascending=False).reset_index(drop=True)
df.head(30)

,CITY,N_OF_ADDS
0,Maribor,1303
1,Ljubljana Moste Polje,953
2,Ljubljana Bežigrad,505
3,Ljubljana Šiška,456
4,Nova Gorica,419
5,Ljubljana Center,367
6,Ljubljana Vič Rudnik,343
7,Domžale,273
8,Kranj,270
9,Celje,251


Vidimo da bi morali biti
- Ljubljana Moste Polje
- Ljubljana Bežigrad
- Ljubljana Šiška
- Ljubljana Center
- Ljubljana Vič Rudnik
- Ljubljana

združeni, zato malce modificirajmo naš števec

In [144]:
d = Counter()
LJUBLJANE = ["Ljubljana Moste Polje", "Ljubljana Bežigrad", "Ljubljana Šiška", "Ljubljana Center", "Ljubljana Vič Rudnik"]
for location in data.City.to_list():
    if len(l := location.split(", ")) == 2:
        city, _ = l
        
        # Če smo v enem izmed sekcij Ljubljane si preprosto zapomnemo Ljubljana
        if city in LJUBLJANE:
            city = "Ljubljana"
    else:
        city = l[0]
    d[city] += 1

In [147]:
df = df_of_counter(d, columns=["CITY", "N_OF_ADDS"]).sort_values('N_OF_ADDS', ascending=False).reset_index(drop=True)
df.head(30)

,CITY,N_OF_ADDS
0,Ljubljana,2836
1,Maribor,1303
2,Nova Gorica,419
3,Domžale,273
4,Kranj,270
5,Celje,251
6,Slovenska Bistrica,229
7,Koper,185
8,Škofljica,183
9,Slovenske Konjice,166


Da bomo pokazali naše sposobnosti z uporabo knjižnice pandas pa pridimo do rezultatov še na lepši način

In [1]:
def to_city(field):
    LJUBLJANE = ["Ljubljana Moste Polje", "Ljubljana Bežigrad", "Ljubljana Šiška", "Ljubljana Center", "Ljubljana Vič Rudnik"]
    if len(l := field.split(", ")) == 2:
        city, _ = l
        
        # Če smo v enem izmed sekcij Ljubljane si preprosto zapomnemo Ljubljana
        if city in LJUBLJANE:
            city = "Ljubljana"
    else:
        city = l[0]
    
    return city

In [6]:
data["City"] = data["City"].map(to_city)
data.groupby("City").size().to_frame(name="N_OF_ADDS").sort_values('N_OF_ADDS', ascending=False)

,N_OF_ADDS
City,
Ljubljana,2836
Maribor,1303
Nova Gorica,419
Domžale,273
Kranj,270
Celje,251
Slovenska Bistrica,229
Koper,185
Škofljica,183
